Data Mininng from X

In [2]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import random
import os

def extract_asin(product_url):
    """
    Extract the ASIN from a standard Amazon product URL.
    """
    match = re.search(r"/dp/([A-Z0-9]{10})", product_url)
    if match:
        return match.group(1)
    return None

def get_reviews_from_page(url, headers, debug=False):
    """
    Extract reviews from a single review page.
    """
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to fetch page:", url, "Status code:", response.status_code)
        return []
    
    html_content = response.text
    # Check for anti-scraping or captcha page
    if "captcha" in html_content.lower() or "are you a robot" in html_content.lower():
        print("Encountered a captcha or anti-scraping page.")
        return []
    
    soup = BeautifulSoup(html_content, 'html.parser')
    reviews = soup.find_all("div", {"data-hook": "review"})
    
    if not reviews and debug:
        snippet = html_content[:500]
        print("No reviews found. HTML snippet:\n", snippet)
    
    review_list = []
    for review in reviews:
        # Extract review title
        title_tag = review.find("a", {"data-hook": "review-title"})
        title = title_tag.text.strip() if title_tag else None

        # Extract review rating (e.g., "5.0 out of 5 stars")
        rating_tag = review.find("i", {"data-hook": "review-star-rating"})
        if not rating_tag:
            rating_tag = review.find("i", {"data-hook": "cmps-review-star-rating"})
        rating = rating_tag.text.strip().split()[0] if rating_tag else None

        # Extract review text
        body_tag = review.find("span", {"data-hook": "review-body"})
        review_text = body_tag.text.strip() if body_tag else None

        # Extract review date
        date_tag = review.find("span", {"data-hook": "review-date"})
        review_date = date_tag.text.strip() if date_tag else None

        # Extract reviewer name
        reviewer_tag = review.find("span", {"class": "a-profile-name"})
        reviewer = reviewer_tag.text.strip() if reviewer_tag else None

        review_list.append({
            "title": title,
            "rating": rating,
            "review_date": review_date,
            "review_text": review_text,
            "reviewer": reviewer
        })
    return review_list

def scrape_amazon_reviews(review_url, max_pages=5, debug=False):
    """
    Scrape reviews from the review URL over multiple pages.
    """
    headers = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/98.0.4758.102 Safari/537.36"),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive"
    }
    all_reviews = []
    for page in range(1, max_pages + 1):
        page_url = f"{review_url}?pageNumber={page}"
        print("Scraping:", page_url)
        reviews = get_reviews_from_page(page_url, headers, debug=debug)
        if not reviews:
            print(f"No reviews found on page {page}.")
            break  # Stop if no reviews are found
        all_reviews.extend(reviews)
        time.sleep(random.uniform(2, 4))  # Polite delay between requests
    return all_reviews

def main():
    # Use your product URL here. For example:
    product_url = ("https://www.amazon.com/BULKSUPPLEMENTS-COM-Creatine-Monohydrate-Powder-"
                   "Unflavored/dp/B00E9M4XFI/ref=pd_hp_d_btf_rpt_sd_biaws_c_7?_encoding=UTF8&dd=gGOKekXGYApE0yMyWt5FUNNGzeC2M1AGI-JCm90QnS4%2C"
                   "&ddc_refnmnt=free&pd_rd_i=B00E9M4XFI&pd_rd_w=RjVuQ&content-id=amzn1.sym.80b400d5-96d8-43db-bf9e-1b83a219e1ac&"
                   "pf_rd_p=80b400d5-96d8-43db-bf9e-1b83a219e1ac&pf_rd_r=X4Z964CYR0WAXS4ARJ44&pd_rd_wg=JVvPl&"
                   "pd_rd_r=741a9954-991b-45c4-a1a0-25c6c4619495&th=1")
    
    # Extract ASIN from the product URL
    asin = extract_asin(product_url)
    if not asin:
        print("ASIN not found in the URL!")
        return

    # Build the review URL using the ASIN
    review_url = f"https://www.amazon.com/product-reviews/{asin}"
    print("Review URL:", review_url)
    
    # Scrape reviews from the first 3 pages; set debug=True for additional output
    reviews = scrape_amazon_reviews(review_url, max_pages=3, debug=True)
    
    if reviews:
        # Create a DataFrame and display the head of the reviews
        df = pd.DataFrame(reviews)
        print("Head of scraped reviews:")
        print(df.head())
        
        # Ensure the output directory exists
        output_dir = r"C:\Users\johne\Desktop\BRAIN\TEMP AMZN Reviews"
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, "reviews.csv")
        
        # Save the DataFrame to CSV
        df.to_csv(output_file, index=False)
        print(f"Reviews saved to {output_file}")
    else:
        print("No reviews scraped.")

if __name__ == "__main__":
    main()


Review URL: https://www.amazon.com/product-reviews/B00E9M4XFI
Scraping: https://www.amazon.com/product-reviews/B00E9M4XFI?pageNumber=1
No reviews found. HTML snippet:
 



<!doctype html><html class="a-no-js" data-19ax5a9jf="dingo">
  <head>
<script type='text/javascript'>var ue_t0=ue_t0||+new Date();</script>
<script type='text/javascript'>
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.cal
No reviews found on page 1.
No reviews scraped.


In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import re
import os

def extract_asin(product_url):
    match = re.search(r"/dp/([A-Za-z0-9]{10})", product_url)
    if match:
        return match.group(1)
    return None

def scrape_reviews_selenium(review_url, max_pages=3):
    options = Options()
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    # Add a realistic user-agent string
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) " 
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")
    
    # Replace this with your actual path to chromedriver.exe
    service = Service(r"C:\WebDriver\bin\chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=options)
    
    all_reviews = []
    wait = WebDriverWait(driver, 15)
    
    for page in range(1, max_pages + 1):
        url = f"{review_url}?pageNumber={page}"
        print("Scraping:", url)
        driver.get(url)
        
        try:
            # Wait until at least one review element is present
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-hook='review']")))
        except Exception as e:
            print(f"Timeout waiting for reviews on page {page}: {e}")
            break
        
        reviews = driver.find_elements(By.CSS_SELECTOR, "div[data-hook='review']")
        if not reviews:
            print(f"No reviews found on page {page}")
            break
        
        for review in reviews:
            try:
                title = review.find_element(By.CSS_SELECTOR, "a[data-hook='review-title']").text
            except Exception:
                title = None
            try:
                rating = review.find_element(By.CSS_SELECTOR, "i[data-hook='review-star-rating']").text.split()[0]
            except Exception:
                rating = None
            try:
                review_text = review.find_element(By.CSS_SELECTOR, "span[data-hook='review-body']").text
            except Exception:
                review_text = None
            try:
                review_date = review.find_element(By.CSS_SELECTOR, "span[data-hook='review-date']").text
            except Exception:
                review_date = None
            try:
                reviewer = review.find_element(By.CSS_SELECTOR, "span.a-profile-name").text
            except Exception:
                reviewer = None
            
            all_reviews.append({
                "title": title,
                "rating": rating,
                "review_date": review_date,
                "review_text": review_text,
                "reviewer": reviewer
            })
    
    driver.quit()
    return all_reviews

def main():
    product_url = "https://www.amazon.com/dp/B0CQMRKRV5"
    asin = extract_asin(product_url)
    if not asin:
        print("ASIN not found!")
        return
    review_url = f"https://www.amazon.com/product-reviews/{asin}"
    print("Review URL:", review_url)
    
    reviews = scrape_reviews_selenium(review_url, max_pages=3)
    
    if reviews:
        df = pd.DataFrame(reviews)
        print("Head of scraped reviews:")
        print(df.head())
        
        output_dir = r"C:\Users\johne\Desktop\BRAIN\TEMP AMZN Reviews"
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, "reviews.csv")
        
        df.to_csv(output_file, index=False)
        print(f"Reviews saved to {output_file}")
    else:
        print("No reviews scraped.")

if __name__ == "__main__":
    main()




Review URL: https://www.amazon.com/product-reviews/B0CQMRKRV5
Scraping: https://www.amazon.com/product-reviews/B0CQMRKRV5?pageNumber=1
Timeout waiting for reviews on page 1: Message: 

No reviews scraped.


In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import re
import os

def extract_asin(product_url):
    match = re.search(r"/dp/([A-Za-z0-9]{10})", product_url)
    if match:
        return match.group(1)
    return None

def scrape_reviews_selenium(review_url, max_pages=3):
    options = Options()
    # Disable headless mode for manual login
   # options.add_argument("--headless")  # Comment this out during initial testing
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) " 
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")
    # Use your personal Chrome profile
    options.add_argument(r"user-data-dir=C:\Users\<YourUsername>\AppData\Local\Google\Chrome\User Data")
    options.add_argument("profile-directory=Default")  # Change if you use another profile
    
    service = Service(r"C:\WebDriver\bin\chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 15)
    
    all_reviews = []
    
    for page in range(1, max_pages + 1):
        url = f"{review_url}?pageNumber={page}"
        print("Scraping:", url)
        driver.get(url)
        
        try:
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-hook='review']")))
        except Exception as e:
            print(f"Timeout waiting for reviews on page {page}: {e}")
            break
        
        reviews = driver.find_elements(By.CSS_SELECTOR, "div[data-hook='review']")
        if not reviews:
            print(f"No reviews found on page {page}")
            break
        
        for review in reviews:
            try:
                title = review.find_element(By.CSS_SELECTOR, "a[data-hook='review-title']").text
            except Exception:
                title = None
            try:
                rating = review.find_element(By.CSS_SELECTOR, "i[data-hook='review-star-rating']").text.split()[0]
            except Exception:
                rating = None
            try:
                review_text = review.find_element(By.CSS_SELECTOR, "span[data-hook='review-body']").text
            except Exception:
                review_text = None
            try:
                review_date = review.find_element(By.CSS_SELECTOR, "span[data-hook='review-date']").text
            except Exception:
                review_date = None
            try:
                reviewer = review.find_element(By.CSS_SELECTOR, "span.a-profile-name").text
            except Exception:
                reviewer = None
            
            all_reviews.append({
                "title": title,
                "rating": rating,
                "review_date": review_date,
                "review_text": review_text,
                "reviewer": reviewer
            })
    
    driver.quit()
    return all_reviews

def main():
    product_url = "https://www.newegg.com/p/N82E16823167063?Item=N82E16823167063"  # Replace with your actual product URL if needed.
    asin = extract_asin(product_url)
    if not asin:
        print("ASIN not found!")
        return
    review_url = f"https://www.newegg.com/Product/ProductReviews.aspx?Item={asin}"
    print("Review URL:", review_url)
    
    reviews = scrape_reviews_selenium(review_url, max_pages=3)
    
    if reviews:
        df = pd.DataFrame(reviews)
        print("Head of scraped reviews:")
        print(df.head())
        
        output_dir = r"C:\Users\johne\Desktop\BRAIN\TEMP AMZN Reviews"
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, "newegg_reviews.csv")
        
        df.to_csv(output_file, index=False)
        print(f"Reviews saved to {output_file}")
    else:
        print("No reviews scraped.")

if __name__ == "__main__":
    main()




ASIN not found!
